In [1]:
import pandas

ImportError: No module named pandas

In [ ]:
titanic = pandas.read_csv('train.csv')
titanic_test1 = pandas.read_csv('test.csv')
titanic_test1.describe()

In [ ]:
titanic_copy = titanic[titanic['Age'].isnull()].copy()
titanic_copy

In [ ]:
titanic.describe()

In [ ]:
#titanic_test.describe()

In [ ]:
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())

In [ ]:
titanic.loc[titanic['Sex'] == 'female', 'Sex'] = 1

In [ ]:
titanic.loc[titanic['Sex'] == 'male', 'Sex'] = 0

In [ ]:
titanic['Embarked'] = titanic['Embarked'].fillna('S')

In [ ]:
titanic.loc[titanic['Embarked'] == 'S', 'Embarked'] = 0


In [ ]:
titanic.loc[titanic['Embarked'] == 'C', 'Embarked'] = 1


In [ ]:
titanic.loc[titanic['Embarked'] == 'Q', 'Embarked'] = 2

# On to machine learning
---

In [ ]:
from sklearn.linear_model import LinearRegression,LogisticRegression

In [ ]:
from sklearn.cross_validation import KFold

In [ ]:
predictors = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [ ]:
alg = LinearRegression()

In [ ]:
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)

In [ ]:
predictions = []

?? Why do we need this initialization? 
# Do predictions(Linear Regressions):
---
KFold function: is to split the original data set into n_folds train/test data sets. For example, here n_folds = 3 mean it will return 3 train(2/3)/test(1/3) data sets. And after the learning you can combine the test results together to get the final prediction results of the entire original data sets.

In [ ]:
for train, test in kf:
    train_predictors = titanic[predictors].iloc[train,:]
    train_target = titanic['Survived'].iloc[train]
    alg.fit(train_predictors, train_target)
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)

# Evaluating Errors
---
Error = $\frac{\#\ of\ correct\ predictions}{\#\ of\ people}$

In [ ]:
import numpy as np

In [ ]:
len(predictions)

In [ ]:
predictions = np.concatenate(predictions, axis=0)

In [ ]:
predictions[predictions > .5] = 1

In [ ]:
predictions[predictions <= .5] = 0

In [ ]:
accuracy = np.sum(predictions[predictions == titanic['Survived']]) / np.size(titanic['Survived'])

In [ ]:
accuracy

# Do predictions(Logistic Regression):

In [ ]:
predicitions_lg = []
alg = LogisticRegression(random_state=1)
import sklearn.cross_validation as cross_validation

In [ ]:
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic['Survived'], cv = 3) 

In [ ]:
np.mean(scores)

# Generating submission file 
---
*  Preprocess test samples

In [ ]:
titanic_test = pandas.read_csv("test.csv")
titanic_test['Age'] = titanic_test['Age'].fillna(titanic['Age'].median())
titanic_test.loc[titanic_test['Sex'] == 'male', 'Sex'] = 0
titanic_test.loc[titanic_test['Sex'] == 'female', 'Sex'] = 1
titanic_test['Embarked'] = titanic_test['Embarked'].fillna('S')
titanic_test.loc[titanic_test['Embarked'] == 'S', 'Embarked'] = 0
titanic_test.loc[titanic_test['Embarked'] == 'C', 'Embarked'] = 1
titanic_test.loc[titanic_test['Embarked'] == 'Q', 'Embarked'] = 2
titanic_test['Fare'] = titanic_test['Fare'].fillna(titanic['Fare'].median())

*  Train Logistic Model

In [ ]:
alg = LogisticRegression()

In [ ]:
alg.fit(titanic[predictors], titanic['Survived'])

In [ ]:
predictions = alg.predict(titanic_test[predictors])

In [ ]:
submission = pandas.DataFrame({
      'PassengerId':titanic_test['PassengerId'],
      'Survived': predictions    
    })

In [ ]:
submission.to_csv('kaggle_submission_lg.csv', index=False)

# Do predictions(Random Forest):
---

In [ ]:
from sklearn import cross_validation

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import numpy as np

In [ ]:
predictors = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
meanscore_array = []

In [ ]:
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=4,
       min_samples_leaf=2)

In [ ]:
scores = cross_validation.cross_val_score(alg, titanic[predictors],
                                         titanic['Survived'], cv=3)

In [ ]:
scores.mean()

In [ ]:
for i in range(30):
    alg = RandomForestClassifier(random_state=1, n_estimators=i+1, min_samples_split=2,
       min_samples_leaf=1)
    scores = cross_validation.cross_val_score(alg, titanic[predictors],
                                         titanic['Survived'], cv=3)
    meanscore_array.append(scores.mean())

In [ ]:
np.argmax(meanscore_array)

## Generating new features
- FamilySize = SibSp + Parch
- NameLength = len(Name)
- Title 
- Family Id,

In [ ]:
titanic['FamilySize'] = titanic['SibSp'] + titanic['Parch']

In [ ]:
titanic['NameLength'] = titanic['Name'].apply(lambda x: len(x))

In [ ]:
import re

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    else:
        return ''
# Another function used to check another way to write the regular expressions
def get_title1(name):
    title_search = re.search(' ([A-Za-z]*)\.', name)
    if title_search:
        return title_search.group(1)
    else:
        return ''

In [ ]:
titles = titanic['Name'].apply(get_title)
titles1 = titanic['Name'].apply(get_title)

In [ ]:
for string, string1 in zip(titles, titles1):
    if(string != string1):
        print string, string1

In [ ]:
pandas.value_counts(titles)

In [ ]:
title_mapping = {'Mr': 1, 'Miss': 2, 'Mrs': 3, 'Master': 4
                , 'Dr': 5, 'Rev': 6, 'Col': 7, 'Major': 8, 'Mlle': 9, 'Countess': 10
                , 'Ms': 11, 'Lady': 12, 'Jonkheer': 13, 'Don': 14, 'Mme': 15, 'Capt': 16, 
                'Sir': 17}

In [ ]:
for k, v in title_mapping.items():
    titles[titles == k] = v

In [ ]:
titanic['Titles'] = titles

In [ ]:
titanic['Titles']

* Family Groups

In [ ]:
import operator
titanic['Name'].head(10)

In [ ]:
family_id_mapping = {}
operator.itemgetter(1)

In [ ]:
def get_family_id(row):
    last_name = row['Name'].split(',')[0]
    family_id = '{0}{1}'.format(last_name, row['FamilySize'])
    if family_id not in family_id_mapping:
        if len(family_id_mapping) == 0:
            current_id = 1
        else:
            # here the operator.itemgetter is used to tell the comparison algo to compare the value/second item in each dict
            # ionary entry, 
            current_id = (max(family_id_mapping.items(), key=operator.itemgetter(1))[1]+1)
        family_id_mapping[family_id] = current_id
    return family_id_mapping[family_id]

In [ ]:
family_ids = titanic.apply(get_family_id, axis=1)

In [ ]:
family_ids[titanic['FamilySize'] < 3] = -1

In [ ]:
pandas.value_counts(family_ids)

In [ ]:
titanic['FamilyId'] = family_ids

## Find out the best features

In [ ]:
import numpy as np

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
predictors = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
             'Fare', 'Embarked', 'FamilySize', 'Titles', 'FamilyId']

In [ ]:
selector = SelectKBest(f_classif, k=5)

In [ ]:
selector.fit(titanic[predictors], titanic['Survived'])

In [ ]:
scores = -np.log10(selector.pvalues_)

In [ ]:
scores

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.bar(range(len(predictors)), scores)
plt.xticks(range(len(predictors)), predictors, rotation='vertical')
plt.show()

In [ ]:
# Pick only the four best features ??????
predictors = ['Pclass', 'Sex', 'Fare', 'Titles']

In [ ]:
alg = RandomForestClassifier(random_state=1, n_estimators=150,
                            min_samples_split=8, min_samples_leaf=4)

In [ ]:
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic['Survived'], cv=3)

In [ ]:
scores.mean()

## Ensembling Method

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
algorithms = [
    [GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3), 
    ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize', 'Titles', 'FamilyId']],
    [LogisticRegression(random_state=1), ['Pclass', 'Sex',
    'Fare', 'FamilySize', 'Titles', 'Age', 'Embarked']     
    ]
]

In [ ]:
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)

In [ ]:
predictions=[]

In [ ]:
for train, test in kf:
    train_target = titanic['Survived'].iloc[train]
    full_test_predictions = []
    for alg, predictors in algorithms:
        alg.fit(titanic[predictors].iloc[train, :], train_target)
        test_predictions = alg.predict_proba(titanic[predictors].iloc[test, :])[:, 1]
        full_test_predictions.append(test_predictions)
    test_predictions = (full_test_predictions[0] + full_test_predictions[1]) / 2
    test_predictions[test_predictions > .5] = 1
    test_predictions[test_predictions <= .5] = 0
    predictions.append(test_predictions)

In [ ]:
predictions = np.concatenate(predictions, axis=0)

In [ ]:
predictions

In [ ]:
accuracy = sum(predictions[predictions == titanic['Survived']]) / len(predictions)

In [ ]:
accuracy

# Make predictions on test dataset

In [ ]:
# First, we'll add titles to the test set.
titles = titanic_test["Name"].apply(get_title)
print(titles)
# We're adding the Dona title to the mapping, because it's in the test set, but not the training set
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6, "Major": 7, "Col": 7, "Mlle": 8, "Mme": 8, "Don": 9, "Lady": 10, "Countess": 10, "Jonkheer": 10, "Sir": 9, "Capt": 7, "Ms": 2, "Dona": 10}
for k,v in title_mapping.items():
    titles[titles == k] = v
titanic_test["Titles"] = titles
# Check the counts of each unique title.
print(pandas.value_counts(titanic_test["Titles"]))

# Now, we add the family size column.
titanic_test["FamilySize"] = titanic_test["SibSp"] + titanic_test["Parch"]

# Now we can add family ids.
# We'll use the same ids that we did earlier.
print(family_id_mapping)

family_ids = titanic_test.apply(get_family_id, axis=1)
family_ids[titanic_test["FamilySize"] < 3] = -1
titanic_test["FamilyId"] = family_ids
titanic_test['NameLength'] = titanic_test['Name'].apply(lambda x: len(x))

In [ ]:
predictors = ["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize", "Titles", "FamilyId"]

algorithms = [
    [GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3), predictors],
    [LogisticRegression(random_state=1), ["Pclass", "Sex", "Fare", "FamilySize", "Titles", "Age", "Embarked"]]
]

full_predictions = []
for alg, predictors in algorithms:
    # Fit the algorithm using the full training data.
    alg.fit(titanic[predictors], titanic["Survived"])
    # Predict using the test dataset.  We have to convert all the columns to floats to avoid an error.
    predictions = alg.predict_proba(titanic_test[predictors].astype(float))[:,1]
    full_predictions.append(predictions)


# The gradient boosting classifier generates better predictions, so we weight it higher.
predictions = (full_predictions[0] * 3 + full_predictions[1]) / 4
predictions[predictions > 0.5] = 1
predictions[predictions <= 0.5] = 0
predictions = predictions.astype(int)
submission = pandas.DataFrame({
    'PassengerId': titanic_test['PassengerId'],
    'Survived': predictions
    })
submission.to_csv('kaggle_gbt.csv', index=False)

In [ ]:
titanic[titanic['Sex'] == 1]

In [ ]:
titanic

# Use xgboost model to do predictions

In [ ]:
import xgboost as xgb

## Turn the original data to DMatrix format

In [ ]:
# Turn the pandas DataFrame to DMatrix format
ftrain = open('./titanic.train.dmatrix', 'w')
ftest = open('./titanic.test.dmatrix', 'w')
for i in range(titanic.shape[0]):
    ftrain.write(str(titanic['Survived'].iloc[i]))
    ftrain.write(' ')
    for j in range(len(predictors)):
        ftrain.write(str(j))
        ftrain.write(':')
        ftrain.write(str(titanic[predictors[j]].iloc[i]))
        ftrain.write(' ')
    ftrain.write('\n')
for i in range(titanic_test.shape[0]):
    ftest.write(str(-1))
    ftest.write(' ')
    for j in range(len(predictors)):
        ftest.write(str(j))
        ftest.write(':')
        ftest.write(str(titanic_test[predictors[j]].iloc[i]))
        ftest.write(' ')
    ftest.write('\n')

In [ ]:
dtrain = xgb.DMatrix('./titanic.train.dmatrix')

In [ ]:
params = {'max_depth':3, 'eta':0.5, 'silent':1, 'objective':'binary:logistic'}
num_round = 20

In [ ]:
t = xgb.cv(params, dtrain, num_round, nfold=4, metrics={'error'}, seed=0)

In [ ]:
t

In [ ]:
bst = xgb.train(params, dtrain, num_round)

In [ ]:
dtest = xgb.DMatrix('./titanic.test.dmatrix')

In [ ]:
preds = bst.predict(dtest)

In [ ]:
labels = dtest.get_label()

In [ ]:
preds

In [ ]:
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0

In [ ]:
preds

In [ ]:
predictions = preds.astype(int)
submission = pandas.DataFrame({
    'PassengerId': titanic_test['PassengerId'],
    'Survived': predictions
    })
submission.to_csv('kaggle_xgboost.csv', index=False)

### Test the round number dependent error rate

In [ ]:
auc_test = []
auc_train = []
No_round = 100
for m in range(No_round):
    result = xgb.cv(params, dtrain, m+1, nfold=4, metrics={'error'}, seed=0)   
    auc_test.append(result['test-error-mean'].iloc[m])
    auc_train.append(result['train-error-mean'].iloc[m])
plt.plot(range(No_round), auc_test, color='red')
plt.plot(range(No_round), auc_train, color='purple')

### Test the depth dependent error rate(5 should be OK)

In [ ]:
result_test = []
result_train = []
No_item = 20
for m in range(No_item):
    params = {'max_depth':m, 'eta':0.5, 'silent':1, 'objective':'binary:logistic'}
    result = xgb.cv(params, dtrain, 20, nfold=4, metrics={'error'}, seed=0)   
    result_test.append(result['test-error-mean'].iloc[m])
    result_train.append(result['train-error-mean'].iloc[m])
print min(result_test)
print result_test.index(min(result_test))
plt.plot(range(No_item), result_test, color='red')
plt.plot(range(No_item), result_train, color='purple')

### Test the eta parameter(0.7 should be OK.)

In [ ]:
result_test = []
result_train = []
No_item = 20
for m in range(No_item):
    params = {'max_depth':3, 'eta':1./float(No_item)*(m), 'silent':1, 'objective':'binary:logistic'}
    result = xgb.cv(params, dtrain, 20, nfold=4, metrics={'error'}, seed=0)   
    result_test.append(result['test-error-mean'].iloc[m])
    result_train.append(result['train-error-mean'].iloc[m])
    print 1/No_item*(m+1)
    print result['test-error-mean'].iloc[m]
print min(result_test)
plt.plot(range(No_item), result_test, color='red')
plt.plot(range(No_item), result_train, color='purple')

In [ ]:
t

In [ ]:
params = {'subsample':1., 'colsample_bytree':1., 'max_depth':2, 'min_child_weight':4, 'eta':0.7, 'silent':1, 'objective':'binary:logistic'}
num_round = 20
xgb.cv(params, dtrain, num_round, nfold=3, metrics={'error', 'auc'}, seed=0) 

In [ ]:
bst = xgb.train(params, dtrain, num_round)
preds = bst.predict(dtest)
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0
predictions = preds.astype(int)
submission = pandas.DataFrame({
    'PassengerId': titanic_test['PassengerId'],
    'Survived': predictions
    })
submission.to_csv('kaggle_xgboost.csv', index=False)

# Take a look at the age distribution

In [ ]:
titanic = pandas.read_csv('train.csv')


In [ ]:
titanic

In [ ]:
titanic_with_age = titanic[~titanic['Age'].isnull()]

In [ ]:
titanic_with_age

In [ ]:
import matplotlib.pyplot as plt
plt.hist(titanic_with_age['Age'], 30, facecolor='blue', alpha=0.5)

In [ ]:
fig, axe = plt.subplots()

In [ ]:
(~titanic['Age'].isnull())[0]

In [ ]:
titanic

In [ ]:
titanic_with_age['Age'][0]